In [2]:
import metpy.calc as mpcalc
from metpy.units import units
import iris
import numpy as np
import sys
sys.path.append('/pf/b/b381639/Analysis')
import fxtian_function as fx
from iris.time import PartialDateTime
import datetime
import cf_units
import iris.coord_categorisation
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import metpy.calc as mpcalc

###############
# plot ERA5
###############

f1='/work/bb1153/b381639/NorESM/QREFHT_150_tropical_ocean.nc'
f2='/work/bb1153/b381639/NorESM/PS_0150_tropical_ocean.nc'
f3='/work/bb1153/b381639/NorESM/SST_150_tropical_ocean.nc'
fQ=xr.open_dataset(f1)
fP=xr.open_dataset(f2)
fSST=xr.open_dataset(f3)
Q=fQ.QREFHT # kg/kg
P=fP.PS
SST=fSST.SST - 273.15 # T in degrees Celsius:



Q=Q*units('kg/kg')
pressure=P*units.Pa
sst=SST*units.degC

In [3]:
fw='/work/bb1153/b381639/NorESM/U10_150_tropical_ocean.nc'
fW=xr.open_dataset(fw)
W10=fW.U10
Varout=xr.zeros_like(W10).rename('wsp10dq')

In [4]:
Ms=mpcalc.saturation_mixing_ratio(pressure, sst)
Qs=mpcalc.specific_humidity_from_mixing_ratio(Ms)


In [5]:
DQ=Qs-Q

In [7]:
WQ=W10*DQ

Varout.data=WQ.data

Varout.attrs['name']='W10DQ'
Varout.attrs['shortName']='wsp10dq'
Varout.attrs['cfVarName']='wsp10dq'

In [10]:
Varout.to_netcdf("/work/bb1153/b381639/NorESM/W10DQ_daily_NorESM.nc")

In [69]:
Es=xr.zeros_like(T)

In [70]:
Es.data =  611.2 * np.exp(17.67*T.data/(T.data + 243.5)) # Es in pasca T in celsius

In [71]:
Es[100,:,50].data

array([      nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan, 

In [72]:
Qs=xr.zeros_like(T)

In [75]:
Qs.data=0.622*Es/P # Qs in kg/kg

In [74]:
Qs[100,:,50].data

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        

In [76]:
import numpy as np
class sounding_df:
    
    def __init__(self, df):
        # this __init__ method is used to create new objects inside the class
        self.p = np.array(df['Pressure'])
        self.temp = np.array(df['Temp']) # in Kelvin
        self.dew = np.array(df['Dewpt']) # in Kelvin
        
    def spec_humidity(self, latent='water'):
        """Calculates SH automatically from the dewpt. Returns in g/kg"""
        # Declaring constants
        self.e0 = 6.113 # saturation vapor pressure in hPa
        # e0 and Pressure have to be in same units
        self.c_water = 5423 # L/R for water in Kelvin
        self.c_ice = 6139 # L/R for ice in Kelvin
        self.T0 = 273.15 # Kelvin
        
        if latent == 'water' or latent == 'Water':
            self.c = self.c_water    # using c for water
        
        else:
            self.c= self.c_ice       # using c_ice for ice, clear state
        
        # saturation vapor not required, uncomment to calculate it (units in hPa becuase of e0)
        #sat_vapor = self.e0 * np.exp((self.c * (self.temp -self.T0))/(self.temp * self.T0)) 
        
        #calculating specific humidity, q directly from dew point temperature
        #using equation 4.24, Pg 96 Practical Meteorolgy (Roland Stull)
#        self.q = (622 * self.e0 * np.exp(self.c * (self.dew - self.T0)/(self.dew * self.T0)))/self.p # g/kg 
        self.q = (622 * self.e0/self.p) * np.exp(self.c * (self.dew - self.T0)/(self.dew * self.T0))
        # 622 is the ratio of Rd/Rv in g/kg
        # where q in the unit of g/kg, e0 and p in the same unit in order to be cancelled out, c, dew and T0 in the unit of K
        return self.q